In [ ]:
import os
import xml.etree.ElementTree as ET
import json
from collections import Counter
from tqdm import tqdm

def create_json_file(object_names_count, folder, filename):
    json_filename = "_".join([f"{obj}{count}" for obj, count in object_names_count.items()]) + ".json"
    json_filepath = os.path.join(folder, json_filename)

    if os.path.exists(json_filepath):
        with open(json_filepath, 'r') as json_file:
            data = json.load(json_file)
    else:
        data = {"fileNames": []}
    
    if filename not in data["fileNames"]:
        data["fileNames"].append(filename)

    with open(json_filepath, 'w') as json_file:
        json.dump(data, json_file, indent=4)

def process_xml_file(xml_path, output_folder):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    filename = root.find('filename').text

    object_names = [obj.find('name').text for obj in root.findall('object')]

    object_names_count = Counter(object_names)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    create_json_file(object_names_count, output_folder, filename)

def iterate_xml_folder(input_folder, output_folder):
    # Get the list of XML files in the folder
    xml_files = [f for f in os.listdir(input_folder) if f.endswith('.xml')]
    
    for xml_file in tqdm(xml_files, desc="Processing XML files"):
        xml_path = os.path.join(input_folder, xml_file)
        process_xml_file(xml_path, output_folder)

if __name__ == "__main__":
    input_folder = '../data/trainval/VOCdevkit/VOC2012/Annotations'
    output_folder = '../data/trainval/relations'
    if os.path.exists(output_folder):
        print('relations exists, exiting')
    else:
        iterate_xml_folder(input_folder, output_folder)


Processing XML files: 100%|██████████| 17125/17125 [04:11<00:00, 68.00it/s]


In [ ]:
import os

def collect_image_filenames(input_folder):
    """Collect all image filenames from the input folder into an array."""
    image_filenames = []
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
            image_filenames.append(filename)
    
    return image_filenames

if __name__ == "__main__":
    input_folder = '../data/trainval/VOCdevkit/VOC2012/SegmentationObject'
    
    image_filenames = collect_image_filenames(input_folder)
    
    print(f"Collected {len(image_filenames)} image filenames:")
    print(image_filenames)


Collected 2913 image filenames:
['2007_000032.png', '2007_000033.png', '2007_000039.png', '2007_000042.png', '2007_000061.png', '2007_000063.png', '2007_000068.png', '2007_000121.png', '2007_000123.png', '2007_000129.png', '2007_000170.png', '2007_000175.png', '2007_000187.png', '2007_000241.png', '2007_000243.png', '2007_000250.png', '2007_000256.png', '2007_000323.png', '2007_000332.png', '2007_000333.png', '2007_000346.png', '2007_000363.png', '2007_000364.png', '2007_000392.png', '2007_000452.png', '2007_000464.png', '2007_000480.png', '2007_000491.png', '2007_000504.png', '2007_000515.png', '2007_000528.png', '2007_000529.png', '2007_000549.png', '2007_000559.png', '2007_000572.png', '2007_000584.png', '2007_000629.png', '2007_000636.png', '2007_000645.png', '2007_000648.png', '2007_000661.png', '2007_000663.png', '2007_000676.png', '2007_000713.png', '2007_000720.png', '2007_000727.png', '2007_000733.png', '2007_000738.png', '2007_000762.png', '2007_000768.png', '2007_000783.png'

In [8]:
import os
import json

def retain_images_in_json(input_folder, images_to_retain):
    """Iterate through JSON files and keep only the images from the fileNames list that appear in images_to_retain array."""
    
    images_to_retain_jpeg = [img.replace('.png', '.jpg') for img in images_to_retain]
    
    json_files = [f for f in os.listdir(input_folder) if f.endswith('.json')]
    
    for json_file in json_files:
        json_path = os.path.join(input_folder, json_file)
        
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        if "fileNames" in data:
            original_count = len(data["fileNames"])
            
            data["fileNames"] = [img for img in data["fileNames"] if img in images_to_retain_jpeg]
            
            retained_count = len(data["fileNames"])
            removed_count = original_count - retained_count
            
            if removed_count > 0:
                with open(json_path, 'w') as f:
                    json.dump(data, f, indent=4)
                print(f"Updated {json_file}: Retained {retained_count} images, Removed {removed_count} images.")
            else:
                print(f"No images removed from {json_file}.")

if __name__ == "__main__":
    input_folder = '../data/relations'
    
    retain_images_in_json(input_folder, image_filenames)


Updated aeroplane1.json: Retained 135 images, Removed 338 images.
Updated aeroplane10.json: Retained 0 images, Removed 1 images.
Updated aeroplane17_person5.json: Retained 0 images, Removed 1 images.
Updated aeroplane1_bird1.json: Retained 0 images, Removed 1 images.
Updated aeroplane1_boat1.json: Retained 1 images, Removed 1 images.
Updated aeroplane1_car1.json: Retained 0 images, Removed 4 images.
Updated aeroplane1_car1_person1.json: Retained 0 images, Removed 1 images.
No images removed from aeroplane1_car1_person2.json.
Updated aeroplane1_car1_person3.json: Retained 0 images, Removed 1 images.
Updated aeroplane1_car2.json: Retained 0 images, Removed 2 images.
Updated aeroplane1_car2_person2.json: Retained 0 images, Removed 1 images.
Updated aeroplane1_car2_person6_motorbike1.json: Retained 0 images, Removed 1 images.
Updated aeroplane1_car4_bus1.json: Retained 0 images, Removed 1 images.
Updated aeroplane1_car8.json: Retained 0 images, Removed 2 images.
Updated aeroplane1_person1.

In [9]:
import os
import json

def delete_empty_jsons(input_folder):
    """Iterate through JSON files and delete those that have no images (empty 'fileNames')."""
    
    json_files = [f for f in os.listdir(input_folder) if f.endswith('.json')]
    
    for json_file in json_files:
        json_path = os.path.join(input_folder, json_file)
        
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        if "fileNames" in data and len(data["fileNames"]) == 0:
            os.remove(json_path)
            print(f"Deleted {json_file} because it contains no images.")

if __name__ == "__main__":
    input_folder = '../data/relations'
    
    delete_empty_jsons(input_folder)


Deleted aeroplane10.json because it contains no images.
Deleted aeroplane17_person5.json because it contains no images.
Deleted aeroplane1_bird1.json because it contains no images.
Deleted aeroplane1_car1.json because it contains no images.
Deleted aeroplane1_car1_person1.json because it contains no images.
Deleted aeroplane1_car1_person3.json because it contains no images.
Deleted aeroplane1_car2.json because it contains no images.
Deleted aeroplane1_car2_person2.json because it contains no images.
Deleted aeroplane1_car2_person6_motorbike1.json because it contains no images.
Deleted aeroplane1_car4_bus1.json because it contains no images.
Deleted aeroplane1_car8.json because it contains no images.
Deleted aeroplane1_person18.json because it contains no images.
Deleted aeroplane1_person1_car1.json because it contains no images.
Deleted aeroplane1_person1_chair1_car1.json because it contains no images.
Deleted aeroplane1_person2.json because it contains no images.
Deleted aeroplane1_pe

In [10]:
import os
import json
import re
from collections import defaultdict

object_pattern = re.compile(r"([a-zA-Z]+)(\d+)")

# Target count for each object
TARGET_COUNT = 256

def calculate_total_object_counts(input_folder):
    """Calculate current counts for each object."""
    object_counts = defaultdict(int)
    
    json_files = [f for f in os.listdir(input_folder) if f.endswith('.json')]
    
    for json_file in json_files:
        json_path = os.path.join(input_folder, json_file)
        
        matches = object_pattern.findall(json_file)
        
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        image_count = len(data.get("fileNames", []))
        
        for obj_name, multiplier in matches:
            multiplier = int(multiplier)
            obj_name = obj_name.capitalize()
            
            object_counts[obj_name] += image_count * multiplier
    
    return object_counts

def remove_images_to_match_target(input_folder, object_counts):
    """Remove images from single-object JSONs for objects that exceed 256 counts."""
    to_remove = {obj: count - TARGET_COUNT for obj, count in object_counts.items() if count > TARGET_COUNT}
    
    json_files = [f for f in os.listdir(input_folder) if f.endswith('.json')]
    
    for json_file in json_files:
        json_path = os.path.join(input_folder, json_file)
        
        matches = object_pattern.findall(json_file)
        
        if len(matches) == 1:
            obj_name, multiplier = matches[0]
            obj_name = obj_name.capitalize() 
            multiplier = int(multiplier)
            
            if obj_name in to_remove:
                with open(json_path, 'r') as f:
                    data = json.load(f)
                
                image_count = len(data.get("fileNames", []))
                remove_count = min(to_remove[obj_name] // multiplier, image_count)
                
                if remove_count > 0:
                    data["fileNames"] = data["fileNames"][:-remove_count]
                    to_remove[obj_name] -= remove_count * multiplier
                    
                    with open(json_path, 'w') as f:
                        json.dump(data, f, indent=4)
                    
                    print(f"Updated {json_file}: Removed {remove_count} images for {obj_name}")
                
                if to_remove[obj_name] <= 0:
                    del to_remove[obj_name]

        if not to_remove:
            break
    
    if to_remove:
        print("\nCould not fully remove images for the following objects (non-single-object JSONs might be needed):")
        for obj_name, remaining in to_remove.items():
            print(f"{obj_name}: {remaining} images still need to be removed.")

if __name__ == "__main__":
    input_folder = '../data/relations'
    
    object_counts = calculate_total_object_counts(input_folder)
    
    remove_images_to_match_target(input_folder, object_counts)

Updated bird1.json: Removed 21 images for Bird
Updated bottle1.json: Removed 46 images for Bottle
Updated bottle10.json: Removed 1 images for Bottle
Updated bottle14.json: Removed 1 images for Bottle
Updated bottle2.json: Removed 7 images for Bottle
Updated bottle3.json: Removed 2 images for Bottle
Updated bottle4.json: Removed 2 images for Bottle
Updated car1.json: Removed 71 images for Car
Updated car2.json: Removed 14 images for Car
Updated car3.json: Removed 3 images for Car
Updated car4.json: Removed 2 images for Car
Updated car5.json: Removed 1 images for Car
Updated car6.json: Removed 2 images for Car
Updated car9.json: Removed 1 images for Car
Updated cat1.json: Removed 30 images for Cat
Updated chair1.json: Removed 23 images for Chair
Updated chair2.json: Removed 3 images for Chair
Updated chair3.json: Removed 2 images for Chair
Updated chair4.json: Removed 3 images for Chair
Updated chair6.json: Removed 2 images for Chair
Updated cow1.json: Removed 28 images for Cow
Updated d

In [11]:
import os
import json
import re
from collections import defaultdict
import statistics
import math

# Regular expression to match object names and capture the number following them
object_pattern = re.compile(r"([a-zA-Z]+)(\d+)")

def calculate_object_stats(input_folder):
    """Iterate through JSON files and calculate total counts, variance, and standard deviation for each object."""
    object_counts = defaultdict(list)  # Dictionary to store lists of counts for each object
    
    # Get the list of JSON files in the input folder
    json_files = [f for f in os.listdir(input_folder) if f.endswith('.json')]
    
    # Iterate through the JSON files
    for json_file in json_files:
        json_path = os.path.join(input_folder, json_file)
        
        # Extract object names and counts from the JSON filename
        matches = object_pattern.findall(json_file)
        
        # Open and load the JSON file
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        # Get the number of images in the JSON file
        image_count = len(data.get("fileNames", []))
        
        # Iterate over the matches from the filename and update the object count lists
        for obj_name, multiplier in matches:
            multiplier = int(multiplier)
            obj_name = obj_name.capitalize()  # Capitalize object names for consistency
            
            # Append the count for this object based on the image count and multiplier
            object_counts[obj_name].append(image_count * multiplier)
    
    # Calculate total counts, variance, and standard deviation for each object
    object_stats = {}
    total_counts_list = []  # List to store the total counts for variance calculation
    for obj_name, counts in object_counts.items():
        total_count = sum(counts)
        variance = statistics.variance(counts) if len(counts) > 1 else 0.0  # Variance is 0 for single counts
        std_dev = math.sqrt(variance)  # Calculate standard deviation
        object_stats[obj_name] = {
            "total_count": total_count,
            "variance": variance,
            "std_dev": std_dev
        }
        total_counts_list.append(total_count)
    
    # Calculate the variance and standard deviation of the total counts across all objects
    total_counts_variance = statistics.variance(total_counts_list) if len(total_counts_list) > 1 else 0.0
    total_counts_std_dev = math.sqrt(total_counts_variance)
    
    return object_stats, total_counts_variance, total_counts_std_dev

if __name__ == "__main__":
    input_folder = '../data/relations'  # Folder containing JSON files
    
    # Calculate the total object counts, variance, and standard deviation
    object_stats, total_counts_variance, total_counts_std_dev = calculate_object_stats(input_folder)
    
    # Output the total counts, variance, and standard deviation for each object
    print("\nObject statistics (total count, variance, and standard deviation):")
    for obj_name, stats in object_stats.items():
        print(f"{obj_name}: Total Count = {stats['total_count']}, Variance = {stats['variance']}, Standard Deviation = {stats['std_dev']}")
    
    # Output the variance and standard deviation of the total counts across all objects
    print("\nVariance and Standard Deviation of Total Counts across all objects:")
    print(f"Total Counts Variance: {total_counts_variance}")
    print(f"Total Counts Standard Deviation: {total_counts_std_dev}")



Object statistics (total count, variance, and standard deviation):
Aeroplane: Total Count = 220, Variance = 743.2753623188406, Standard Deviation = 27.26307690483304
Boat: Total Count = 232, Variance = 176.8825396825397, Standard Deviation = 13.299719533980396
Car: Total Count = 316, Variance = 4.958130927596576, Standard Deviation = 2.226686086451473
Person: Total Count = 1492, Variance = 12.639254460734174, Standard Deviation = 3.5551729157291594
Bus: Total Count = 237, Variance = 54.15488018702513, Standard Deviation = 7.358999944763224
Bicycle: Total Count = 197, Variance = 30.714971751412428, Standard Deviation = 5.542108962426887
Bottle: Total Count = 260, Variance = 14.325271059216014, Standard Deviation = 3.7848739819465607
Tvmonitor: Total Count = 199, Variance = 27.535116074052308, Standard Deviation = 5.247391358956592
Chair: Total Count = 490, Variance = 8.26213043751177, Standard Deviation = 2.8743921857519323
Sofa: Total Count = 209, Variance = 14.298754295532646, Standa

In [ ]:
import os
import json
import random
import albumentations as A
import cv2

augmentations = {
    "noise": A.GaussNoise(p=1.0),
    "color_jitter": A.RandomBrightnessContrast(p=1.0),
    "blur": A.GaussianBlur(blur_limit=7, p=1.0),
    "sharpen": A.Sharpen(alpha=(0.2, 0.5), p=1.0),
}

def augment_images_to_target(relation_folder, input_folder, output_folder, target_count, actual_count, class_name):
    num_images_to_generate = target_count - actual_count
    if num_images_to_generate <= 0:
        print(f"No need for augmentation, target count {target_count} already met.")
        return

    collected_images = []
    json_files = [f for f in os.listdir(relation_folder) if f.endswith('.json') and class_name.lower() in f.lower()]
    
    for json_file in json_files:
        json_path = os.path.join(relation_folder, json_file)
        with open(json_path, 'r') as f:
            data = json.load(f)
            collected_images.extend(data.get("fileNames", []))
    
    if not collected_images:
        print(f"No images found for class '{class_name}' in the relation folder.")
        return

    os.makedirs(output_folder, exist_ok=True)

    images_generated = 0
    while images_generated < num_images_to_generate:
        for img_name in collected_images:
            if images_generated >= num_images_to_generate:
                break

            img_path = os.path.join(input_folder, img_name)
            image = cv2.imread(img_path)
            if image is None:
                print(f"Error: Could not read image {img_path}")
                continue
            
            aug_name, aug_choice = random.choice(list(augmentations.items()))
            augmented_image = aug_choice(image=image)["image"]
            
            original_name = os.path.splitext(img_name)[0]
            
            output_img_name = f"{original_name}-{aug_name}.jpg"
            output_img_path = os.path.join(output_folder, output_img_name)
            cv2.imwrite(output_img_path, augmented_image)
            
            images_generated += 1

    print(f"Generated {images_generated} new images for class '{class_name}' and saved to '{output_folder}'.")



# Example usage:
augment_images_to_target(
    relation_folder='../data/relations',
    input_folder='../data/trainval/VOCdevkit/VOC2012/JPEGImages',
    output_folder='../data/trainval/VOCdevkit/VOC2012/AugmentedImages',
    target_count=256,
    actual_count=209,
    class_name='sofa'
)

c:\Users\naang\anaconda3\envs\DL_test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\naang\anaconda3\envs\DL_test\lib\site-packages\pydantic\main.py:212: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


Generated 47 new images for class 'sofa' and saved to '../data/VOCdevkit/VOC2012/AugmentedImages'.
